In [13]:
#web_scrapping 
from bs4 import BeautifulSoup
import requests
import pandas as pd




In [31]:
url = 'https://swnsdigital.com/uk/2023/02/these-are-the-top-20-unwanted-fruit-and-vegetables-from-bagged-salad-leaves-to-bananas/'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")


# Find all <li> elements
list_items = soup.find_all('li')


# Initialize an empty list to store data
data = []

# Iterate through each <li> element
for li in list_items:
    # Find <span> elements within the <li> element
    span_elements = li.find_all('span')
    # Extract the text content of each <span> element
    span_texts = [span.text for span in span_elements]
    # Add the extracted data to the list
    data.append(span_texts)

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)  # Adjust column names as needed

# Print the DataFrame
df = df.dropna().reset_index(drop=True)
df= df.rename(columns={0:'top_fruit_veg_wasted'})
df.to_csv('../source/top_fruit_veg_wasted.csv',index=False)
